In [6]:
import json
import random
import uuid
import time
from confluent_kafka import Producer
import fastavro
from fastavro.schema import load_schema
import io

# Avro schema (hardcoded)
avro_schema = {
    "type": "record",
    "name": "CreditTransaction",
    "namespace": "com.fraud.detection",
    "fields": [
        {
            "name": "client_id",
            "type": "string"
        },
        {
            "name": "transaction_id",
            "type": "string"
        },
        {
            "name": "type",
            "type": ["string", "null"],
            "default": None
        },
        {
            "name": "amount",
            "type": ["double", "null"],
            "default": None
        },
        {
            "name": "oldbalanceOrg",
            "type": ["double", "null"],
            "default": None
        },
        {
            "name": "newbalanceOrig",
            "type": ["double", "null"],
            "default": None
        }
    ]
}

# Kafka Producer Configuration
KAFKA_BROKER = "host.docker.internal:9092"  # Replace with your Kafka broker
TOPIC = "raw_credit_data"  # Kafka topic to send data to

# Initialize Kafka producer
conf = {
    'bootstrap.servers': KAFKA_BROKER,
    'acks': 'all',  # Wait for all brokers to acknowledge
}

producer = Producer(conf)

# Generate random test data
def generate_test_data():
    client_id = f"client_{random.randint(1000, 9999)}"
    transaction_id = str(uuid.uuid4())
    transaction_type = random.choice(["TRANSFER","DEBIT","CASH_OUT","CASH_IN"])
    amount = random.uniform(1.0, 10.0)
    old_balance = random.uniform(0.0, 5000.0)
    new_balance = old_balance - amount if transaction_type == "purchase" else old_balance + amount

    return {
        "client_id": client_id,
        "transaction_id": transaction_id,
        "type": transaction_type,
        "amount": amount,
        "oldbalanceOrg": old_balance,
        "newbalanceOrig": new_balance
    }

# Function to serialize data into Avro format
def serialize_avro(data, schema):
    with io.BytesIO() as buf:
        writer = fastavro.writer(buf, schema, [data])
        return buf.getvalue()

# Produce data to Kafka
def produce_to_kafka():
    for _ in range(1):  # Adjust the number of records to send
        test_data = generate_test_data()
        avro_data = serialize_avro(test_data, avro_schema)
        
        # Produce message to Kafka
        producer.produce(topic=TOPIC, key=test_data["transaction_id"], value=avro_data)
        print(f"Sent data: {test_data}")
        producer.flush()

    

# Start the producer
if __name__ == "__main__":
    produce_to_kafka()


Sent data: {'client_id': 'client_1067', 'transaction_id': 'e96ec732-64f4-47d5-a536-e2e1e822e59a', 'type': 'TRANSFER', 'amount': 5.970366120317026, 'oldbalanceOrg': 4154.109938797385, 'newbalanceOrig': 4160.080304917702}
